## Melting curve (Tm) analysis.  

### creating the environment, importing necessary packages
                    

### Import data from 100's of raw data.txt file. Do the QC and remove the artifacts. Make dataframe for ARGs specific Tm's for all the data in the database.


In [ ]:
import glob
import os
files_folder = "/data/**/"
csv_files = glob.glob(os.path.join(files_folder, "1*.txt"))
all_projects_tm = []
for f in csv_files:
    df = pd.read_csv(f, delimiter='\t', names =['Row', 'Column', 'Assay', "Sample", "Conc", "Ct","Tm", "Efficiency"], header=None)
    path=os.path.dirname(f)
    df = df.iloc[1: , :]
    df['filename'] = os.path.basename(f)
    df['project_number'] = os.path.basename(path)
    df['Ct'] = pd.to_numeric(df['Ct'],errors='coerce')
    df['Tm'] = pd.to_numeric(df['Tm'],errors='coerce')
    df['Efficiency'] = pd.to_numeric(df['Efficiency'],errors='coerce')
    df['project_number'] = pd.to_numeric(df['project_number'], errors='coerce')
    df['Row'] = pd.to_numeric(df['Row'])
    df = pd.DataFrame(df)

# drop all Ct NaN values which means no amplification
    df.dropna(subset = ["Ct"], inplace=True)
#Getting first 2 letters of sample name
    df['Sample_number'] = df['Sample'].str[:3]
   
#all ARGs in each text file
    All_assay = df['Assay'].unique()
    i= 0
    while i < len(All_assay):
#all samples per ARGs per text file
        single_assay = df.loc[df.Assay == All_assay[i], :]
        count= single_assay['Sample_number'].value_counts()
#QC steps
#exlude samples that  have only 1 out of 3 technical replicate working
        more_than_1_rep = single_assay[single_assay["Sample_number"].isin(count[count > 1].index)]   
#exclude Ct value >= 28
        tm_lessthan_28_Ct =  more_than_1_rep[more_than_1_rep["Ct"] < 28]   
#dataframe for all tm for samples with more than 1 out of 3 techical replicate working, and Ct values less than 28   
        all_projects_tm.append(tm_lessthan_28_Ct)
        i=i+1

: 

##### Make dataframe for tm of all data in the database.

In [ ]:
df_all_projects_tm = pd.concat(all_projects_tm)

#Making df of samples only, getting rid of controls which are un-numbered
search = "1","2", "3", "4","5", "6", "7","8", "9"
df_Ct_tm_of_samples_only  =  df_all_projects_tm [df_all_projects_tm ["Sample_number"].str.startswith(search)]

: 

#### Tm analysis for each ARGs for all the projects in the database 

In [ ]:
df_all_projects_tm_stat =  df_all_projects_tm.groupby('Assay').agg({'Tm': ['mean', 'median','min', 'max', 'std','count']})

#importing gene list
df_gene_names= pd.read_excel ('/gene_names.xlsx')
df_gene_names.rename(columns = {"assay":'Assay'}, inplace = True)


#merging count top 50 list with gene names and present in decending order of count
df_all_projects_tm_stat_with_gene_name= df_gene_names.merge(df_all_projects_tm_stat, on='Assay')  

writer = pd.ExcelWriter('/Tm_analysis.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet
df_all_projects_tm_stat_with_gene_name.to_excel(writer, index= False)  
# Close the Pandas Excel writer and output the Excel file.
writer.save()


### Country specific Top 50 ARGs for each sample type

In [ ]:
#create empty DFs
dicts_countries_per_sample_type = {}
df_all_assay= {}
df_all_assay_with_signal= {}
abun_data_most_detected_assay= {}
df_abun_data_most_detected_assay= {}
df_abun_data_most_detected_assay_with_gene_name= {}
df_sample_ID_= {}

countries_list = df_samples['country'].unique()
sample_list = [df_sampleID_manure, df_sampleID_sediment, df_sampleID_sludge, df_sampleID_soil, df_sampleID_stool, df_sampleID_surface_water, df_sampleID_waste_water, df_sampleID_other ]

j= 0
while j < len(sample_list):
    for i in countries_list:
            df = sample_list[j]
            dicts_countries_per_sample_type[i] = df.loc[df['country'] == (i)]
    
#get all assay (genes) for each country using sample_id in abundance dataframe
            df_all_assay[i] = df_abun.merge( dicts_countries_per_sample_type[i], on= 'sample_id')
#get rid of NULL values of ralative abudances    
            df_all_assay_with_signal[i] = df_all_assay[i].dropna(subset=['relative'])
#for each assay show general statistic for relative abundance, and show the top 50 according to relative abundance, count means total time gene is assayed equals total samples assayed
            df_abun_data_most_detected_assay[i] = df_all_assay_with_signal[i].groupby('assay').agg({'relative': ['mean', 'min', 'max', 'count']}).relative.sort_values(by=['mean'],ascending=False).nlargest(50, 'mean')
#merging mean top 50 list with gene names and present in decending order of mean of relative abundance
            df_abun_data_most_detected_assay_with_gene_name[i]= df_gene_names.merge(df_abun_data_most_detected_assay[i], on='assay').sort_values(by=['mean'],ascending=False)
# Create a Pandas Excel writer using XlsxWriter as the engine.
    file_name = df.type[1] + "- Top 50 most detected genes for each country.xlsx"
    writer = pd.ExcelWriter('Documents/'+ file_name, engine='xlsxwriter')
    for i in countries_list:
# Write each dataframe to a different worksheet.
        df_abun_data_most_detected_assay_with_gene_name[i].to_excel(writer, sheet_name= str(i), index= False)
# Close the Pandas Excel writer and output the Excel file.
    writer.save()
    j=j+1